In [3]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

2024-03-02 17:23:44.974177: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-02 17:23:45.032788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 17:23:45.032843: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 17:23:45.034422: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-02 17:23:45.044980: I tensorflow/core/platform/cpu_feature_guar

In [5]:
# load the train.csv file into a pandas dataframe
df = pd.read_csv('TRAIN.csv')
#convert the dates columns to a single column as "Date " with associated sales
df = pd.melt(df, id_vars=['Item code', 'Category', 'State'], var_name='date', value_name='sales')

# Convert the 'Date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')

print(df.shape)

(6463692, 5)


In [6]:
print('Min date from train set: %s' % df['date'].min().date())
print('Max date from train set: %s' % df['date'].max().date())

# date before 3 months from the last date in the train set
date_90_days_ago = df['date'].max() - pd.DateOffset(days=90)
print('Date 90 days ago from start predict date: %s' % date_90_days_ago.date())

Min date from train set: 2017-01-29
Max date from train set: 2022-02-22
Date 90 days ago from start predict date: 2021-11-24


In [7]:
# choose dataframe with date 150 days before the last date
date_149_days_ago = df['date'].max() - pd.DateOffset(days=149)
df = df[df['date'] > date_149_days_ago]
print('Min date from chosen dataset: %s' % df['date'].min().date())
print(df.shape)

Min date from chosen dataset: 2021-09-27
(520308, 5)


### Convert data to training time series format

In [8]:
# Group by date, item code, category and state and calculate the mean sales
df = df.sort_values('date').groupby(['Item code','Category','State','date'], as_index=False)
df = df.agg({'sales':['mean']})
df.columns = ['Item code','Category','State','date','sales']

In [9]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    # cols stores the shifted dataframes
    shifted_dataframes, shifted_column_names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        # we are shifting the dataframe rows by i steps and are storing the new dataframes and there column names in a list
        shifted_dataframes.append(data.shift(i))
        shifted_column_names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    shifted_dataframes.append(data)
    shifted_column_names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    shifted_dataframes.append(data.shift(-lag))
    shifted_column_names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(shifted_dataframes, axis=1)
    agg.columns = shifted_column_names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### Convert the timeseries data into lagged features data with target sales after 90 days

In [10]:
window = 59
lag = 90
series = series_to_supervised(df, window=window, lag=lag,dropnan=False)
print(series.shape)
series.head()

(520308, 305)


,Item code(t-59),Category(t-59),State(t-59),date(t-59),sales(t-59),Item code(t-58),Category(t-58),State(t-58),date(t-58),sales(t-58),...,Item code(t),Category(t),State(t),date(t),sales(t),Item code(t+90),Category(t+90),State(t+90),date(t+90),sales(t+90)
0,None,None,None,NaT,NaN,None,None,None,NaT,NaN,...,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-09-27,0.0,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-12-26,1.0
1,None,None,None,NaT,NaN,None,None,None,NaT,NaN,...,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-09-28,0.0,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-12-27,0.0
2,None,None,None,NaT,NaN,None,None,None,NaT,NaN,...,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-09-29,0.0,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-12-28,0.0
3,None,None,None,NaT,NaN,None,None,None,NaT,NaN,...,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-09-30,0.0,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-12-29,0.0
4,None,None,None,NaT,NaN,None,None,None,NaT,NaN,...,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-10-01,0.0,ANTIBIOTIC_001,ANTIBIOTIC,MH,2021-12-30,0.0


### Avoid the scenario  when we are trying to predict the sale of a different product using different product

In [11]:
last_Item_code = 'Item code(t-%d)' % window
last_category = 'Category(t-%d)' % window
last_state = 'State(t-%d)' % window

series = series[(series['Item code(t)'] == series[last_Item_code])]
series = series[(series['Category(t)'] == series[last_category])]
series = series[(series['State(t)'] == series[last_state])]

print(series.shape)
# series.head()

(314280, 305)


In [12]:
print('Min date from train set: %s' % series['date(t)'].min().date())
print('Max date from train set: %s' % series['date(t)'].max().date())

print(series['date(t)'].min().date()+pd.DateOffset(days=lag))

Min date from train set: 2021-11-25
Max date from train set: 2022-02-22
2022-02-23 00:00:00


### Drop the extra Item code, Category and state columns as they are same across all 30 lagged features

In [13]:
columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Item code', 'Category', 'State', 'date']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Item code', 'Category', 'State','date']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['Item code(t)', 'Category(t)', 'State(t)','date(t)'], axis=1, inplace=True)
print(series.shape)
series.head()

(314280, 61)


,sales(t-59),sales(t-58),sales(t-57),sales(t-56),sales(t-55),sales(t-54),sales(t-53),sales(t-52),sales(t-51),sales(t-50),...,sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t),sales(t+90)
59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,...,1.0,6.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
60,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,...,6.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0
61,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0
62,0.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0
63,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,1.0,0.0


In [14]:
# # Now apply normalization to the data
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(series)

In [15]:
labels_col = 'sales(t+%d)' % lag
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

# convert the labels and the features to numpy arrays
X = series.values
y = labels.values
print(X.shape)
print(y.shape)

(314280, 60)
(314280,)


In [16]:
X = X.reshape((X.shape[0], X.shape[1], 1))
print('Validation set shape', X.shape)

Validation set shape (314280, 60, 1)


In [17]:
epochs = 10
batch = 256
lr = 0.0003
adam = optimizers.Adam(lr)

2024-03-02 17:23:57.047580: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22319 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:98:00.0, compute capability: 8.6


In [18]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mse', optimizer=adam)
model_lstm.summary()

2024-03-02 17:23:57.415041: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10451 (40.82 KB)
Trainable params: 10451 (40.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# load the model from disk
from keras.models import load_model
model_lstm = load_model('best_model.hdf5')

In [20]:
lstm_pred = model_lstm.predict(X)

9822/9822 [==============================] - 229s 23ms/step


In [31]:
# load the train.csv file into a pandas dataframe
df = pd.read_csv('TRAIN.csv')
#convert the dates columns to a single column as "Date " with associated sales
df = pd.melt(df, id_vars=['Item code', 'Category', 'State'], var_name='date', value_name='sales')

# Convert the 'Date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')

In [32]:
date_90_days_ago_from_first_prediction = df['date'].max() - pd.DateOffset(days=90)
print('Date 90 days ago from start predict date: %s' % date_90_days_ago_from_first_prediction.date())
df= df[df['date'] > date_90_days_ago_from_first_prediction]
print('Min date from train set: %s' % df['date'].min().date())

Date 90 days ago from start predict date: 2021-11-24
Min date from train set: 2021-11-25


In [33]:
# offset each date by 90 days to get the real date
df['date'] = df['date'] + pd.to_timedelta(90, unit='d')
print('Min date from train set: %s' % df['date'].min().date())

# now replace the sales column with the predicted values
df['sales'] = lstm_pred

# convert datetime to string with just the date
df['date'] = df['date'].dt.strftime('%m-%d-%y')

# now we need to convert the dataframe to the original format
df = df.pivot_table(index=['Item code', 'Category', 'State'], columns='date', values='sales').reset_index()

# save the dataframe to a csv file
df.to_csv('submission.csv', index=False)

Min date from train set: 2022-02-23
